# Homework 4: Sentence Segmentation

Sentence segmentation is a crucial, yet under-appreciated, part of any text processing pipeline. In any real-world setting, text will not come to you pre-chunked into sentences, and if your pipeline involves any sentence-level analysis (parsing, translation, entity & co-reference extraction, etc.) the first thing you'll need to do will be split things up into sentences. This is a deceptively challenging problem, as punctuation can be used in multiple ways. Consider the following sentence (borrowed from Kyle Gorman's [excellent discussion of the subject](http://www.wellformedness.com/blog/simpler-sentence-boundary-detection/)): 

> _"Rolls-Royce Motor Cars Inc. said it expects its U.S. sales to remain steady at about 1,200 cars in 1990."_ 

This sentence features several periods, but only one of them (the last one) represents a valid sentence break. Sentences can also have embedded clauses or quotation marks, some of which may superficially look like a sentence boundary: 

> _He says the big questions–“Do you really need this much money to put up these investments? Have you told investors what is happening in your sector? What about your track record?–“aren’t asked of companies coming to market._

There are many ways to approach this particular problem; it can be approached using rules, which themselves may be encoded as regular expressions, or (more usefully) it can be modeled as a machine learning problem. Specifically, as a binary classification problem, where candidate sentence boundaries (i.e., punctuation marks from the closed class of English punctuation that can represent a sentence boundary) are either _positive_ examples (actual sentence boundaries) or _false_ (not sentence boundaries). 

In this assignment, you will develop and test your own sentence boundary detection algorithm. This assignment is shorter than our previous assignments, and is also much more open-ended. By this point in the class, you have been exposed to several different families of technique for text classification and analysis, and have hopefully begun to build up some intuitions. For this assignment, you can choose whatever methods you like: log-linear models, rule-based methods, some sort of neural network, it's up to you!

## 0. Setup & data exploration

In [2]:
from importlib import reload
from sklearn.metrics import classification_report, accuracy_score

In [3]:
from hw4_utils import data
from hw4_utils import classifier

We've provided a version of the WSJ section of the Penn Treebank to use for training and evaluation purposes. See `data/wsj_sbd/README.md` for a description of where this data came from and how it was assembled.

The `hw4_utils.data` module has a useful method for reading in data (`file_as_string`) and another utility function to extract candidate boundaries. Let's use these to explore our data a little bit.

In [13]:
dev = data.file_as_string("data/wsj_sbd/dev.txt")
train = data.file_as_string("data/wsj_sbd/train.txt")
test = data.file_as_string("data/wsj_sbd/test.txt")

In [14]:
dev[:100]

'Pierre Vinken, 61 years old, will join the board as a nonexecutive director Nov. 29.\nMr. Vinken is c'

Each of these is a string containing the text, with one input sentence per line. Note that newline characters are still present, which will be important as this will provide us with our ground truth.

Remember, we _only_ will use the `test` split for our final evaluation. For data exploration, feature engineering, and for tuning hyperparameters, use the `dev` split.

The first and most important thing to find out when doing classification is what your class probabilities are, so let's look into that. The `load_candidates()` function in the `hw4_utils.data` module will read an input data split, identify candidate boundaries, and extract information about the candidates context that we may want to use for feature engineering. One of the attributes it will extract is whether or not the candidate is actually a true sentence break (`is_true_break`). Obviously, this would only be useful at training time, as in the "real world" we wouldn't know the answer. :-)

We will use `load_candidates()` to answer our question about class balance:

In [17]:
from collections import Counter

Counter([c.is_true_break for c in data.load_candidates(dev)])

Counter({False: 1766, True: 5769})

As is often the case, our classes are quite imbalanced! Let's load in our ground truth labels:

In [16]:
y_dev = [o.is_true_break for o in data.load_candidates(dev)]

What else can we find out about our candidate breaks?

In [18]:
candidates = list(data.load_candidates(dev[:10000])) # for now, just get candidates from the first 10k characters
first_can = candidates[0]

In [19]:
print("original context: ",first_can.orig_obs)
print("punctuation mark: ", first_can.punctuation_mark)
print("token to the left: ", first_can.left_token)
print("token to the right: ", first_can.right_token)

original context:  d as a nonexecutive director Nov. 29. Mr. Vinken is chairman of El
punctuation mark:  .
token to the left:  Nov
token to the right:  29


Take a look at the the `Observation` namedtuple in `hw4_utils.data` for more details about the various attributes that you can find out about each possible sentence boundary.

## 1. Baselines

We will begin by looking at three different baselines. First, we will 

### Baseline #1: Majority class

Since our data are quite imbalanced, what if we always treat every candidate boundary as though it was a true boundary?

In [20]:
y_hat_baseline = [classifier.baseline_classifier(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_baseline))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00      1766
        True       0.77      1.00      0.87      5769

   micro avg       0.77      0.77      0.77      7535
   macro avg       0.38      0.50      0.43      7535
weighted avg       0.59      0.77      0.66      7535



C:\Users\steve\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Baseline #2: Next-token capitalization

What if we say that a candidate is a boundary if the following token is capitalized?

In [22]:
y_hat_next_tok_cap = [classifier.next_tok_capitalized_baseline(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_next_tok_cap))

              precision    recall  f1-score   support

       False       0.60      0.40      0.48      1766
        True       0.83      0.92      0.87      5769

   micro avg       0.80      0.80      0.80      7535
   macro avg       0.71      0.66      0.67      7535
weighted avg       0.78      0.80      0.78      7535



### Baseline #3: Punkt

Using the [NLTK implementation](http://www.nltk.org/_modules/nltk/tokenize/punkt.html) of [Punkt (Kiss & Strunk, 2006)](https://www.mitpressjournals.org/doi/pdf/10.1162/coli.2006.32.4.485):

(Note: You'll need to have downloaded the NLTK pre-trained Punkt model for this baseline to work)

In [23]:
y_hat_punkt = [classifier.punkt_baseline(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_punkt))

              precision    recall  f1-score   support

       False       0.92      0.43      0.59      1766
        True       0.85      0.99      0.91      5769

   micro avg       0.86      0.86      0.86      7535
   macro avg       0.88      0.71      0.75      7535
weighted avg       0.87      0.86      0.84      7535



### Conclusion

None of these are especially inspiring in their performance... can you do better?

## 2. Your turn!

### Part 1: Now it's your turn.

Your mission, in this assignment, is to implement at least two additional approaches. The approaches must be different in terms of the features you choose to use, classification approach, or both, but beyond that limitation you are free to do whatever you like as long as it is your own work (i.e., do not download and use an already-existing sentence boundary detector tool). You may, however, replicate an existing algorithm, though please be very careful not about re-using other people's code. 

For a discussion of features that may prove useful, and of other approaches to this task that you may wish to replicate or build off of, consult the bibliography discussed in [Gorman (2014)](http://www.wellformedness.com/blog/simpler-sentence-boundary-detection/); note that the specific sentence boundary detector described on that page uses a small set of highly useful features, but you will want to go beyond simply replicating his system.

***Deliverable***: The code for your ≥ two additional classifiers, along with a brief writeup (≈1 page) of what you built and why you chose those specific approaches and features. 

### Part 2: Evaluation & Error analysis

After building and describing your additional classifiers, evaluate their performance against the `test` partition of the data as shown above. Write a brief summary of their relative performance, and include an _error analysis_. Examine cases where your classifiers failed to predict correctly, or disagreed with one another, and see if you can identify any patterns that might explain where they were going awry.

***Deliverable***: Your written summary, as well as any relevant data tables.